# Stuff to do
* in final app, give them just the allowed options (dropdown menu, for example)

In [1]:
from cmp import *
import pdir
%matplotlib notebook
#np.seterr(invalid='raise')
np.set_printoptions(threshold=np.nan)

In [2]:
# Inputs
eq = np.isclose
# Lattice vectors (3 vectors of length 3)
a = 1
b = 2
a1 = np.array([1, 0, 0])
a2 = np.array([0, 1, 0])
a3 = np.array([0, 0, 1])
theta = 80*np.pi/180

# Array of basis vectors
basis = np.array([[0,0,0],[0.5,0.5,0],[0.5,0,0.5],[0,0.5,0.5]])
# Colors for each of the basis vectors
blargh = ('r', 'r','b','b')
# Size multiplier for each of the atoms. Default is 1
sizes = (2,2,1,1)
verbose = True


# Gridline type:
# Soft: Lines along cartesian axes. Takes into account nonequal lattice spacing
# LatticeVectors: Lines along the latticevectors (only on lattice points)
GridType = "lattice"

# Limit type:
# individual: Sets the limits as max(nx*a1,ny*a2,nz*a3), so we include nx unitcells in the a1 direction, etc.
# sum: Sets the limits r_min = n_min*[a1 a2 a3] and likewise for n_max
LimType = "dynamic"
Maxs = [2,2,2]
Mins = [0,0,0]

LatticeType = "conventional fcc"

#Lattice(lattice_name = LatticeType, colors = blargh, sizes = sizes, max_ = Maxs, verbose=True)
#Reciprocal(lattice_name=LatticeType, indices=(1,1,0))

In [4]:
k0 = np.linspace(0,np.pi, 100)
x0 = np.cos(k0)
V0 = 1

def fx(x, k, V0):
    X = np.sqrt(2)*x
    sin0 = np.sin(X)
    cos0 = np.cos(X)
    f = cos0 - V0*sin0/(X) - np.cos(k)
    return f

def fx2(x, k, V0):
    return np.cos(np.sqrt(2)*x) - V0*np.sin(np.sqrt(2)*x)/(np.sqrt(2)*x) - np.cos(k)

def fxprime(x, k, V0):
    scale = np.sqrt(2)
    X = scale * x
    sin0 = np.sin(X)
    cos0 = np.cos(X)
    fprime = -scale * sin0 + V0 * scale * sin0 / (2 * x**2) - V0 * cos0/x
    return fprime

def fxprime2(x, k, V0):
    return -np.sqrt(2) * np.sin(np.sqrt(2)*x) + V0 * np.sqrt(2) * np.sin(np.sqrt(2) * x)/(2 * x**2) - V0 * np.cos(np.sqrt(2)*x) / x

def newtn(x, k, V0):
    scale = np.sqrt(2)
    X = scale * x
    sin0 = np.sin(X)
    cos0 = np.cos(X)
    next_ = (V0 * scale* sin0 / 2 + x * (np.cos(k) - cos0)) * x / ((x**2 - V0/2)*scale*sin0 + V0*cos0*x)
    return next_

def newtn_calc(x, k, V0, iterations = 10):
    for i in range(iterations):
        x -= newtn(x, k, V0)
    return x

E = newtn_calc(x0, k0, 1)

E0 = np.linspace(-3*np.pi/8, np.pi, 100)
#E00 = np.sqrt(E0)
#k = np.arccos(np.cos(np.sqrt(2)*E00) - 1*np.sin(np.sqrt(2)*E00)/(np.sqrt(2)*E00))

/home/niko/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in sqrt
